# Import

read later https://towardsdatascience.com/set-your-jupyter-notebook-up-right-with-this-extension-24921838a332


In [68]:
from gtts import gTTS
import os
import wikipedia
import os
from google.cloud import translate
#from ClassText2Speech import speak_chinese
from xpinyin import Pinyin
import jieba
import jieba.posseg as pos
import jieba.analyse as ana
import sys
from importlib import *
from collections import Counter
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import time
#import matplotlib.pyplot as plt
#%matplotlib inline

from PIL import Image
import requests
from io import BytesIO

In [2]:
# List of common words that should be excluded from analysis
jieba_stop_words = ["、","。","〈","〉","《","》","一","一些","一何","一切","一则",
                    "一方面","一旦","一来","一样","一般","一转眼","七","万一","三",
                    "上","上下","下","不","不仅","不但","不光","不单","不只","不外乎",
                    "不如","不妨","不尽","不尽然","不得","不怕","不惟","不成","不拘",
                    "不料","不是","不比","不然","不特","不独","不管","不至于","不若",
                    "不论","不过","不问","与","与其","与其说","与否","与此同时","且",
                    "且不说","且说","两者","个","个别","中","临","为","为了","为什么",
                    "为何","为止","为此","为着","乃","乃至","乃至于","么","之","之一",
                    "之所以","之类","乌乎","乎","乘","九","也","也好","也罢","了","二",
                    "二来","于","于是","于是乎","云云","云尔","五","些","亦","人","人们",
                    "人家","什","什么","什么样","今","介于","仍","仍旧","从","从此","从而",
                    "他","他人","他们","他们们","以","以上","以为","以便","以免","以及",
                    "以故","以期","以来","以至","以至于","以致","们","任","任何","任凭",
                    "会","似的","但","但凡","但是","何","何以","何况","何处","何时","余外",
                    "作为","你","你们","使","使得","例如","依","依据","依照","便于","俺",
                    "俺们","倘","倘使","倘或","倘然","倘若","借","借傥然","假使","假如",
                    "假若","做","像","儿","先不先","光是","全体","全部","八","六","兮",
                    "共","关于","关于具体地说","其","其一","其中","其二","其他","其余",
                    "其它","其次","具体地说","具体说来","兼之","内","再","再其次","再则",
                    "再有","再者","再者说","再说","冒","冲","况且","几","几时","凡","凡是",
                    "凭","凭借","出于","出来","分","分别","则","则甚","别","别人","别处","别是",
                    "别的","别管","别说","到","前后","前此","前者","加之","加以","即","即令","即使",
                    "即便","即如","即或","即若","却","去","又","又及","及","及其","及至","反之",
                    "反而","反过来","反过来说","受到","另","另一方面","另外","另悉","只","只当",
                    "只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是",
                    "可见","各","各个","各位","各种","各自","同","同时","后","后者","向","向使",
                    "向着","吓","吗","否则","吧","吧哒","含","吱","呀","呃","呕","呗","呜","呜呼",
                    "呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱们","咳","哇",
                    "哈","哈哈","哉","哎","哎呀","哎哟","哗","哟","哦","哩","哪","哪个","哪些",
                    "哪儿","哪天","哪年","哪怕","哪样","哪边","哪里","哼","哼唷","唉","唯有","啊",
                    "啐","啥","啦","啪达","啷当","喂","喏","喔唷","喽","嗡","嗡嗡","嗬","嗯","嗳",
                    "嘎","嘎登","嘘","嘛","嘻","嘿","嘿嘿","四","因","因为","因了","因此","因着",
                    "因而","固然","在","在下","在于","地","基于","处在","多","多么","多少","大",
                    "大家","她","她们","好","如","如上","如上所述","如下","如何","如其","如同","如是",
                    "如果","如此","如若","始而","孰料","孰知","宁","宁可","宁愿","宁肯","它","它们",
                    "对","对于","对待","对方","对比","将","小","尔","尔后","尔尔","尚且","就","就是",
                    "就是了","就是说","就算","就要","尽","尽管","尽管如此","岂但","己","已","已矣","巴",
                    "巴巴","年","并","并且","庶乎","庶几","开外","开始","归","归齐","当","当地","当然",
                    "当着","彼","彼时","彼此","往","待","很","得","得了","怎","怎么","怎么办","怎么样",
                    "怎奈","怎样","总之","总的来看","总的来说","总的说来","总而言之","恰恰相反","您","惟其",
                    "慢说","我","我们","或","或则","或是","或曰","或者","截至","所","所以","所在","所幸",
                    "所有","才","才能","打","打从","把","抑或","拿","按","按照","换句话说","换言之","据",
                    "据此","接着","故","故此","故而","旁人","无","无宁","无论","既","既往","既是","既然",
                    "日","时","时候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有关",
                    "有及","有时","有的","望","朝","朝着","本","本人","本地","本着","本身","来","来着","来自",
                    "来说","极了","果然","果真","某","某个","某些","某某","根据","欤","正值","正如","正巧",
                    "正是","此","此地","此处","此外","此时","此次","此间","毋宁","每","每当","比","比及",
                    "比如","比方","没奈何","沿","沿着","漫说","焉","然则","然后","然而","照","照着","犹且",
                    "犹自","甚且","甚么","甚或","甚而","甚至","甚至于","用","用来","由","由于","由是",
                    "由此","由此可见","的","的确","的话","直到","相对而言","省得","看","眨眼","着","着呢",
                    "矣","矣乎","矣哉","离","秒","竟而","第","等","等到","等等","简言之","管","类如",
                    "紧接着","纵","纵令","纵使","纵然","经","经过","结果","给","继之","继后","继而",
                    "综上所述","罢了","者","而","而且","而况","而后","而外","而已","而是","而言","能",
                    "能否","腾","自","自个儿","自从","自各儿","自后","自家","自己","自打","自身","至",
                    "至于","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如",
                    "莫若","虽","虽则","虽然","虽说","被","要","要不","要不是","要不然","要么","要是",
                    "譬喻","譬如","让","许多","论","设使","设或","设若","诚如","诚然","该","说","说来",
                    "请","诸","诸位","诸如","谁","谁人","谁料","谁知","贼死","赖以","赶","起","起见",
                    "趁","趁着","越是","距","跟","较","较之","边","过","还","还是","还有","还要","这",
                    "这一来","这个","这么","这么些","这么样","这么点儿","这些","这会儿","这儿","这就是说",
                    "这时","这样","这次","这般","这边","这里","进而","连","连同","逐步","通过","遵循",
                    "遵照","那","那个","那么","那么些","那么样","那些","那会儿","那儿","那时","那样",
                    "那般","那边","那里","都","鄙人","鉴于","针对","阿","除","除了","除外","除开",
                    "除此之外","除非","随","随后","随时","随着","难道说","零","非","非但","非徒","非特",
                    "非独","靠","顺","顺着","首先","︿","！","＃","＄","％","＆","（","）","＊","＋",
                    "，","０","１","２","３","４","５","６","７","８","９","：",
                    "；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥","\n", " ",'借傥', '兼',
                    '前', '唷', '啪', '啷', '喔', '天', '始', '漫', '然', '特', '竟', '莫', '见', '设', '达',
                   "!" ,"#", "$", "%","&", "(",")","+", "-","/","=","'", ",", "?","他們","並且",
                   "擁有","擁有","一個","我們"]

Print out the stop words and become halfway familiar with these.  These are the most common words, so much so that our program will ignore them completely.  Therefore, I am going to keep a crib sheet on hand for these words.

## Connect API

In [3]:
#api key for google api
## you'll need to get your own API key using google cloud console and replace the path name
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/db/Desktop/APIKeys/LearnChinese.json"

In [4]:
#Instantiate Frequently Used 
p = Pinyin()

In [5]:
# Jeiba can only work with utf-8 encoding so we need to create a file that is constantly rewritten
f = open("myfile.txt", "w", encoding="utf-8")
f.write("")
f.close()

# Google Translate API  [Read-The-Docs](https://www.geeksforgeeks.org/python-docstrings/ )

## Simple English to Chinese Translation Function

In [6]:
#translates English to Chinese for the initial search of wikipedia in language being learned
def english2chinese(topic, display=False):
    """ Translates a word or words from English into Traditional Mandarin"""
    # Instantiates a client
    translate_client = translate.Client()
    text = topic
    # Defines the target language
    target = 'zh-TW'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    if display == True:
        print(text_)
        return text_
    else:
        return text_

### Sanity Check

In [7]:
# sanity check
# english2chinese('Batman') #'蝙蝠俠'

## Simple Chinese to English Translation Function

In [8]:
# translates Chinese to English
def chinese2english(topic):
    # Instantiates a client
    translate_client = translate.Client()
    text = topic
    # Defines the target language
    target = 'en'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    if display == True:
        print(text_)
        return text_
    else:
        return text_

### Sanity Check

In [9]:
# Sanity Check!
#english2chinese('Batman') #'蝙蝠俠'

## Speech-to-Text

## English To Chinese 

In [10]:
def e2c_say_anything():
    translate_client = translate.Client()
    text = input('What word would you like to say in Chinese(Mandarin)：')
    # Defines the target language
    target = 'zh-TW'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    
    #add pinyin
    pinyin_with_numbers = p.get_pinyin(text_, tone_marks='numbers')
    tts = gTTS(text=text_, lang='zh-TW')
    tts.save("good.mp3")
    
    print("Input: {}".format(text))
    print("Traditional Chinese Translation: {}".format(text_))
    print("Pinyin Translation with tone numbers: {}".format(pinyin_with_numbers))
    return os.system("mpg321 good.mp3")

### Sanity Check

In [11]:
# Sanity Check
# e2c_say_anything()
# What word would you like to say in Chinese(Mandarin)：Batman
# Input: Batman
# Traditional Chinese Translation: 蝙蝠俠
# Pinyin Translation with tone numbers: bian1-fu2-xia2

## Chinese To English

In [12]:
def c2e_say_anything(text):
    translate_client = translate.Client()
    text = text
    # Defines the target language
    target = 'en'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    
    
    tts = gTTS(text=text_, lang='en')
    tts.save("good.mp3")
    
    print("Input: {}".format(text))
    print("Output: {}".format(text_))
    return os.system("mpg321 good.mp3")
    

### Sanity Check

In [13]:
#Sanity Check
c2e_say_anything('蝙蝠俠')
# What Chinese word would you like to say in English?：蝙蝠俠
# Input: 蝙蝠俠
# Output: Batman

Input: 蝙蝠俠
Output: Batman


0

# Wikipedia Search

In [14]:
def search_wikipedia(topic, display=False,results=3):
    wikipedia.set_lang('zh-TW')
    search_phrase = english2chinese(topic)
    pages = wikipedia.search(search_phrase, results=3)
    if display == True:
        print('the pages in native language', pages)
        return pages
    else:
        return pages

In [15]:
# Sanity Check
search_wikipedia('Batman')
# ['蝙蝠俠',
#  '蝙蝠俠：開戰時刻',
#  '蝙蝠俠：高譚騎士',
#  '樂高蝙蝠俠電影',
#  '蝙蝠侠 (动画影集)',
#  '蝙蝠侠归来',
#  '蝙蝠俠對超人：正義曙光',
#  '永远的蝙蝠侠',
#  '蝙蝠侠与罗宾 (电影)',
#  '蝙蝠俠：黑暗騎士歸來']

['蝙蝠俠', '蝙蝠俠：開戰時刻', '蝙蝠俠：高譚騎士']

In [16]:
def get_content(topic):
    """Takes a topic Parameter and returns the wikipedia page that best matches that topic in Chinese"""
    wikipedia.set_lang('zh-TW')
    search_phrase = english2chinese(topic)
    wiki_page = wikipedia.page(search_phrase,preload=True)
    return wiki_page.content

In [17]:
get_content('Batman')

'蝙蝠俠（英語：Batman）是一名出現於DC漫畫的虛構超級英雄角色，由鲍勃·凱恩和比爾·芬格創作，角色首次登場於《偵探漫畫》第27期（1939年5月）。最初被稱為「蝙蝠人」，後來還有「黑暗騎士」、「世界最偉大的偵探」等其他稱號。\n蝙蝠俠的真實身份為布魯斯·韋恩（英語：Bruce Wayne），是一位美國億萬富翁、花花公子、企業家和慈善家。十岁時目睹了雙親被歹徒殺害，使得他立下為向罪犯復仇的誓言。長大後，布鲁斯開始訓練自己的身體和知識，並開發了蝙蝠戰衣以打擊犯罪。蝙蝠俠主要在高譚市行動，並透過其他配角來協助他，如管家阿爾弗萊德·潘尼沃斯、搭檔羅賓、警察局長詹姆斯·高登 和蝙蝠女孩。蝙蝠俠常對付各種職業罪犯，如小丑、企鵝人、謎語人、猫女、急凍人、雙面人、拉斯·阿勒古尔、稻草人、毒藤女、喪鐘、泥面人以及班恩。蝙蝠俠和多數的超級英雄不同，他不擁有任何超能力，但常利用智力、科學知識、偵探技能、高科技裝備、財富、武術、百折不撓的毅力和恐嚇來對付敵人。\n蝙蝠俠走紅後不久，DC便推出它的獨立作品《蝙蝠俠》（1940年），在几十年的连载中，在不同作者的创作下有著各種不同的塑造和背景。1960年代末，電視劇《蝙蝠俠》結束後人氣停滞了多年。在各创作者努力下，最终在弗兰克·米勒的1986年漫画《蝙蝠侠：黑暗骑士归来》、阿兰·摩尔的《蝙蝠侠：致命玩笑》中回归他最初的黑暗风格。1989年的华纳兄弟的蝙蝠侠系列电影则将蝙蝠侠在流行文化中的地位提升到了一个新的高度。\n成為了美國漫画的文化代表之一後，蝙蝠俠已改編成电视和電影，並發行了各種玩具與電子遊戲。其角色也成了許多精神科醫生和學者研究的對象。2011年5月，在IGN的前100名的漫畫書英雄中蝙蝠俠排行第二。2015年，時代公司旗下的網站FanSided將他列為「漫畫史上最偉大的50位超級英雄」的第一名。蝙蝠俠在影视媒体上已由演員路易斯·威爾遜、羅伯特·洛厄里、亞當·韋斯特、米高·基頓、凱文·康羅伊、方·基默、喬治·克隆尼、克里斯汀·貝爾和班·艾佛列克饰演過。\n\n\n== 出版歷史 ==\n\n\n=== 角色诞生 ===\n1938年初，由於之前超人的成功，讓国家联合出版公司的漫畫部編輯開始尋求更多的超級英雄出版品。鲍勃·凯恩因此而創造出了蝙蝠俠。他的合作者比尔·芬格回忆说:“有一天凯恩说他有一个叫‘蝙蝠侠’的角色的想法，他想让我看

In [18]:
def wikipedia_page_choices(topic, language='en'):
    pages = search_wikipedia(topic)
    translate_client = translate.Client()
    target_en = language
    print('Titles of wikipedia page')
    for page in pages:
            english_translation = translate_client.translate(page, target_language=target_en)
            english_text = english_translation['translatedText']
            print(english_text)
        

In [19]:
#sanity check
wikipedia_page_choices('Batman')
# #The following pages are in Chinese but the titles are translated into English
# Batman
# Batman: The moment of war
# Batman: Gotham Knight
# Lego Batman Movie
# Batman (Animation Album)
# Batman returns
# Batman vs. Superman: The Light of Justice
# Forever Batman
# Batman and Robin (movie)
# Batman: Return of the Dark Knight

Titles of wikipedia page
Batman
Batman: The moment of war
Batman: Gotham Knight


In [20]:
# rerun this cell to clear the myfile.txt without having to rerun all previous cells
f = open("myfile.txt", "w", encoding="utf-8")
f.write("")
f.close()

In [21]:
def getChinese(content):
    content = content.decode("utf-8") # convert content from str to unicode
    filtrate = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
    content = filtrate.sub(r'', content) # remove all non-Chinese characters
    content = content.encode("utf-8") # convert unicode back to str
    return content

In [89]:
corpus = []
from IPython.core.display import Image, display
def combine_wikipedia_pages(query, suggestion=True):
    translated_query = english2chinese(query)
    titles = wikipedia.search(translated_query, results = 2)
    filter('', titles)
    for title in titles:
        webpage = wikipedia.page(title)
        content = webpage.content
        image = webpage.images[0:]
#         content = getChinese(content)
        #remove links
        #content = re.sub("http://[a-zA-z./\d]*"," ",content)
        # remove emojis
        #content = re.sub("\[.{0,12}\]"," ",content)
        #remove digits
        #content = re.sub("\d","",content)
        # remove elipses
        #content = re.sub("\.*"," ",content)
        #extract and remove all english characters
        english = re.findall("[a-z]+",content)
        content = re.sub("[a-z]+","",content)
        print("page added to content")
        f = open("myfile.txt", "a")
        f.write(str(content))
        f.close()
        picture =Image(url= str(image[0]))
        corpus.append(content)
        
    print("process finished, all content stored in myfile.txt")
    display(picture)

In [90]:
#sanity check
combine_wikipedia_pages('Harvard')

page added to content
page added to content
process finished, all content stored in myfile.txt


In [91]:
#read in the text file
with open('myfile.txt','r')as myfile:
    data=myfile.read()

In [92]:
#segement and save 
seg_list = jieba.cut(data, cut_all=False)

In [93]:
seg_list = [str(word) for word in seg_list if not str(word) in jieba_stop_words]

In [94]:
new_text = ("".join(seg_list))

In [95]:

f=open('myfile.txt','w')
f.write(new_text)
f.close()

In [96]:
top20 = jieba.analyse.extract_tags(new_text, topK=20, withWeight=True)

In [97]:
vocab = pd.DataFrame(top20, columns=['Chinese','Weight'])

In [98]:
def get_pinyin(word, tone_marks='numbers'):
    return p.get_pinyin(word, tone_marks='numbers')

In [99]:
vocab['Pinyin']= vocab['Chinese'].apply(get_pinyin)
    

In [100]:
vocab['English'] = vocab['Chinese'].apply(chinese2english)

In [101]:
vocab

,Chinese,Weight,Pinyin,English
0,哈佛,0.211667,ha1-fo2,Harvard
1,香港,0.151063,xiang1-gang3,Hong Kong
2,大學,0.086492,da4-xue2,the University
3,學院,0.071467,xue2-yuan4,College
4,學生,0.048736,xue2-sheng1,student
5,課程,0.043920,ke4-cheng2,course
6,發展,0.040645,fa1-zhan3,development of
7,圖書館,0.040260,tu2-shu1-guan3,library
8,開始,0.032555,kai1-shi3,Start
9,成為,0.031399,cheng2-wei4,become


In [102]:
def pronounce_chinese(text_):
    tts = gTTS(text=text_, lang='zh-TW')
    tts.save("good.mp3")
    return os.system("mpg321 good.mp3")

In [103]:
pronounce_chinese("開始")

0

Clear 

In [104]:
# rerun this cell to clear the myfile.txt without having to rerun all previous cells
f = open("myfile.txt", "w", encoding="utf-8")
f.write("")
f.close()

# Working MVP

In [105]:
def application(topic):
    combine_wikipedia_pages(topic)
    #read in the text file
    with open('myfile.txt','r')as myfile:
        data=myfile.read()
    
    #segement and save 
    seg_list = jieba.cut(data, cut_all=False)
    seg_list = [str(word) for word in seg_list if not str(word) in jieba_stop_words]
    new_text = ("".join(seg_list))
    #overwrite myfile.txt without stopwords
    f=open('myfile.txt','w')
    f.write(new_text)
    f.close()
    #20 most important vocabulary words to know
    top20 = jieba.analyse.extract_tags(new_text, topK=20, withWeight=True)
    #store to dataframe
    vocab = pd.DataFrame(top20, columns=['Chinese','Weight'])
    #add pinyin to dataframe
    vocab['Pinyin']= vocab['Chinese'].apply(get_pinyin)
    #add english translation to dataframe
    vocab['English'] = vocab['Chinese'].apply(chinese2english)
    return vocab

In [106]:
df = application('Hong Kong')

page added to content
page added to content
process finished, all content stored in myfile.txt


In [107]:
df.drop_duplicates(subset='English',keep='first', inplace=True)

In [108]:
df

,Chinese,Weight,Pinyin,English
0,香港,0.347577,xiang1-gang3,Hong Kong
1,英國,0.061816,ying1-guo2,United Kingdom
2,中國,0.051066,zhong1-guo2,China
3,經濟,0.050170,jing1-ji4,economic
4,香港政府,0.045494,xiang1-gang3-zheng4-fu3,Hong Kong government
5,發展,0.043899,fa1-zhan3,development of
6,行政,0.041730,xing2-zheng4,administrative
7,中國大陸,0.033148,zhong1-guo2-da4-lu4,China Mainland
8,開始,0.032252,kai1-shi3,Start
9,時期,0.032252,shi2-qi1,period


Bring in cc-cedict dictionary

In [135]:
dictionary = pd.read_csv('cedict.csv',sep='|',index_col=0)

In [136]:
dictionary.head()

,Unnamed: 1,pā,percent (Tw).
%,,,
21三體綜合症,21三体综合症,èr shí yī sān tǐ zōng hé zhèng,trisomy; Down's syndrome.
3C,NaN,sān C,"abbr. for computers, communications, and consu..."
3P,NaN,sān P,(slang) threesome.
3Q,NaN,sān Q,(Internet slang) thank you (loanword).
502膠,502胶,wǔ líng èr jiāo,cyanoacrylate glue.


In [ ]:
for

In [243]:
for index,row in df.iterrows():
    print(row['English'])
    #time.sleep(10)
    pronounce_chinese(row['Chinese'])

Christmas
celebrate
Birthday
Santa Claus
holiday
gift
Christmas Island
1225
Holiday
Christian
Christ
religion
Jesus
Australia
Christmas Eve
new Year
festival


In [ ]:
if 節日

In [201]:
pronounce_chinese("克拉克")

0